In [ ]:
import PyPDF2
import re
import os
import time
import pandas as pd
import numpy as np
import warnings
warnings.simplefilter("ignore", UserWarning)
%cd ..

a:\Monthy\5_2023\Crawl_buffett


In [ ]:

def convert_pdf_to_text(pdf_path):
    text = ""
    with open(pdf_path, "rb") as file:
        pdf_reader = PyPDF2.PdfReader(file)
        num_pages = len(pdf_reader.pages)
        
        for page_number in range(num_pages):
            page = pdf_reader.pages[page_number]
            text += page.extract_text()
    
    return text


def find_by_re(text2):
    text2 = text2.replace(' ', '')  
    numbers = re.findall(r'\d{1,3}(?:,\d{3})*', text2)
    filtered_numbers = [int(number.replace(',', '')) for number in numbers if int(number.replace(',', '')) > 3000]
    if len(filtered_numbers) == 4:
        lst_data_of_time = [filtered_numbers[0], filtered_numbers[2]]
    elif len(filtered_numbers) == 2:
        lst_data_of_time = filtered_numbers
    else:
        lst_data_of_time = [np.nan,np.nan]
    return lst_data_of_time

def find_row(text):
    text1 = text[text.find('期末発行済株式数') :text.find('期末発行済株式数')+125]
    matches = re.finditer('株', text1)
    positions = [match.start() for match in matches]

    text2 = text1[:positions[-1]+1]
    text2 = text2.replace('③', '\n③').replace(' ,', ',')
    return find_by_re(text2)


def get_data_from_pdf(id_company, year, quy):
    for file in os.listdir(f'{id_company}/PDF'):
        if file.startswith(f'{year}_{quy}'):
            file_name = file
            text = convert_pdf_to_text(f'{id_company}/PDF/{file_name}')
            try:
                lst_data_of_time = find_row(text)
            except:
                lst_data_of_time = ['Bug',"Bug"]
            return lst_data_of_time
    return ['Không có file','Không có file']


get_data_from_pdf(1333, 2014, 'Q1')


In [1]:
import pandas as pd

In [3]:
df = pd.read_csv('A:\Phong\Crawl_buffett\List_company_23052023 - Listing.csv')

In [4]:
for i in df.index:
    print(df['Symbol'][i])

1301
1332
1333
1375
1376
1377
1379
1380
1381
1382
1383
1384
1400
1401
1407
1414
1417
1418
1419
1420
1429
1430
1431
1433
1434
1435
1436
1438
1439
1443
1444
1446
1447
1450
1451
1491
1514
1515
1518
1605
1662
1663
1711
1712
1716
1717
1718
1719
1720
1721
1723
1724
1726
1730
1736
1739
1743
1757
1758
1762
1764
1766
1768
1770
1773
1775
1776
1780
1783
1786
1787
1788
1789
1793
1795
1798
1799
1801
1802
1803
1805
1807
1808
1810
1811
1812
1813
1814
1815
1820
1821
1822
1826
1827
1828
1833
1835
1840
1841
1844
1847
1848
1850
1852
1853
1860
1861
1866
1867
1870
1871
1873
1878
1879
1882
1884
1885
1887
1888
1890
1893
1897
1898
1899
1904
1905
1909
1911
1914
1921
1925
1926
1928
1929
1930
1934
1938
1939
1941
1942
1944
1945
1946
1948
1949
1950
1951
1952
1954
1959
1960
1961
1963
1964
1965
1966
1967
1968
1969
1971
1972
1973
1975
1976
1979
1980
1981
1982
1992
1994
1997
2001
2002
2003
2004
2009
2053
2055
2060
2107
2108
2109
2112
2114
2117
2120
2121
2122
2124
2127
2130
2134
2136
2138
2139
2146
2148
2150
2152
2153


In [9]:
def get_volume(id_company):
    df_volume = pd.DataFrame(columns=['time', 'vol1', 'vol2'])
    df = pd.read_csv(f'{id_company}/docs/link.csv')
    for quy in ['Q1', 'Q2', 'Q3', 'Q4']:
        for id in df.index:
            year = df[f'Year'][id]
            df_volume.loc[(len(df_volume))] = [f'{year}_{quy}'] + get_data_from_pdf(id_company, year, quy)
    # return df_volume
    df_volume.to_csv(f'{id_company}/docs/volume.csv', index=False)

get_volume(5486)